In [ ]:
## start ##
import os
import pandas as pd
import numpy as np
from numpy import average


os.chdir('/Users/danielmederer/Data Storage Folder')
os.getcwd()




In [ ]:
df_trait = pd.read_csv("/Users/danielmederer/Data Storage Folder/gap_filled_final_7targets_with_ranges.csv", sep = ",")
df_trait.head()



In [ ]:
df_trait = df_trait[['Species', 'chlc_mygcm2', 'nitrogenc_gm2', 'LMA_gm2', 'EWT_mgcm2', 
                     'antc_gcm2', 'caroc_gcm2', 'LAI_m2m2']]
df_trait

In [ ]:
###IMPORTANT###
# RUN EVERYTHING FROM HERE TRICE, ONCE FOR GRASS, ONCE FOR FOREST AND ONCE FOR SHRUBLAND

In [ ]:
df_species = pd.read_csv("splot_subset_grass_balanced.csv", sep = ",") #ONE

In [ ]:
df_species = pd.read_csv("splot_subset_forest_balanced.csv", sep = ",") #TWO

In [ ]:
df_species = pd.read_csv("splot_subset_shrubland_balanced.csv", sep = ",") #THREE

In [ ]:

df_species = df_species[['Species', 'PlotObservationID', 
                      'Relative_cover']]
df_species

In [ ]:
#df_species = pd.read_csv("Sylt_species_covers_cor.csv", sep = ";")
#df_species

In [ ]:
df_plot = (df_species.pivot_table(columns='PlotObservationID', index=['Species'], 
                       values='Relative_cover').reset_index().rename_axis(None, axis = 1))
df_plot

In [ ]:
df_plot = df_plot.fillna(0)


In [ ]:
# make a string list of species names for the plot data

species_list = df_species['Species'].unique().tolist()

In [ ]:
# for each string, select all values in the trait column of the trait data

trait_lists = df_trait.groupby('Species')['chlc_mygcm2', 'nitrogenc_gm2', 'LMA_gm2', 
                                          'EWT_mgcm2', 'antc_gcm2', 'caroc_gcm2', 'LAI_m2m2'
                                          ].agg(["mean", "std"])


# get mean and standard deviation of every list in the dataframe

trait_lists.columns = ['chl_mean', 'chl_std', 'nit_mean', 'nit_std','lma_mean', 'lma_std', 
                       'ewt_mean', 'ewt_std', 'ant_mean', 'ant_std', 'car_mean', 'car_std', 
                       'lai_mean', 'lai_std', 
                      ]

#trait_lists.columns = ['ant_mean', 'ant_std', 'car_mean', 'car_std', 'chl_mean', 'chl_std',
 #                          'lma_mean', 'lma_std', 'ewt_mean', 'ewt_std', 'lai_mean', 'lai_std']


trait_lists


In [ ]:
#trait_lists2.agg(random.normal(loc=0.0, scale=1.0, size=50))

# next step: write column names of new distribution dataframe
cols = ['Species','chl_smpl', 'nit_smpl', 'lma_smpl', 'ewt_smpl', 'ant_smpl', 'car_smpl',
       'lai_smpl']

data = []
# Group by species
for spec, values in trait_lists.groupby('Species'):
    # Loop over rows for each species generating 50 values per trait
    for i, r in values.iterrows():
        data.append([spec,             
        abs(np.random.choice(np.random.normal(r['chl_mean'], r['chl_std'], 50))), 
        abs(np.random.choice(np.random.normal(r['nit_mean'], r['nit_std'], 50))),
        abs(np.random.choice(np.random.normal(r['lma_mean'], r['lma_std'], 50))),
        abs(np.random.choice(np.random.normal(r['ewt_mean'], r['ewt_std'], 50))),
        abs(np.random.choice(np.random.normal(r['ant_mean'], r['ant_std'], 50))),
        abs(np.random.choice(np.random.normal(r['car_mean'], r['car_std'], 50))),
        abs(np.random.choice(np.random.normal(r['lai_mean'], r['lai_std'], 50))),
       ])   
        #data.append([spec,             
        #np.random.choice(np.random.normal(r['chl_mean'], r['chl_std'], 50)), 
        #np.random.choice(np.random.normal(r['nit_mean'], r['nit_std'], 50)),
        #np.random.choice(np.random.normal(r['lma_mean'], r['lma_std'], 50)),
        #np.random.choice(np.random.normal(r['ewt_mean'], r['ewt_std'], 50)),
        #np.random.choice(np.random.normal(r['ant_mean'], r['ant_std'], 50)),
        #np.random.choice(np.random.normal(r['car_mean'], r['car_std'], 50)),
        #np.random.choice(np.random.normal(r['lai_mean'], r['lai_std'], 50)),
       #])   
        
ddf = pd.DataFrame(data, columns = cols)

# Combine plot df with normal distribution df, using NAs for missing species

combined_df = pd.merge(df_plot, ddf, on="Species", how="left")

combined_df


In [ ]:
combined_df["ewt_smpl"].sample(3)

In [ ]:
len(combined_df.columns)

In [ ]:
# 2. get weighted averages for each plot in the plot data

# write a loop that goes through every plot column , 'car_avg', 'chl_avg', 'lma_avg', 'ewt_avg', 'lai_avg'

cols = ['plot', 'chl_avg', 'nit_avg', 'lma_avg', 'ewt_avg', 'ant_avg', 'car_avg', 'lai_avg', ]
avg_list = []

# -7 depends on number of traits!
for plot in combined_df.columns[1:(len(combined_df.columns) - 7)]:
    
    avg_list.append([plot, round(np.ma.average(np.ma.MaskedArray(combined_df['chl_smpl'], 
                                                            mask=np.isnan(combined_df['chl_smpl'])), 
                             weights = combined_df[plot]),5),
                    round(np.ma.average(np.ma.MaskedArray(combined_df['nit_smpl'], 
                                                            mask=np.isnan(combined_df['nit_smpl'])), 
                             weights = combined_df[plot]),5),
                    round(np.ma.average(np.ma.MaskedArray(combined_df['lma_smpl'], 
                                                            mask=np.isnan(combined_df['lma_smpl'])), 
                             weights = combined_df[plot]),5),
                    round(np.ma.average(np.ma.MaskedArray(combined_df['ewt_smpl'], 
                                                            mask=np.isnan(combined_df['ewt_smpl'])), 
                             weights = combined_df[plot]),5),
                    round(np.ma.average(np.ma.MaskedArray(combined_df['ant_smpl'], 
                                                            mask=np.isnan(combined_df['ant_smpl'])), 
                             weights = combined_df[plot]),5),
                    round(np.ma.average(np.ma.MaskedArray(combined_df['car_smpl'], 
                                                            mask=np.isnan(combined_df['car_smpl'])), 
                             weights = combined_df[plot]),5),
                     round(np.ma.average(np.ma.MaskedArray(combined_df['lai_smpl'], 
                                                            mask=np.isnan(combined_df['lai_smpl'])), 
                             weights = combined_df[plot]),5)]
                   )
                   
    
w_avg_per_plot = pd.DataFrame(avg_list, columns = cols)


w_avg_per_plot  

In [ ]:

min(w_avg_per_plot["nit_avg"])


In [ ]:
print(w_avg_per_plot[(w_avg_per_plot[w_avg_per_plot.columns] > 0).all(axis=1)])
df_end = w_avg_per_plot[(w_avg_per_plot[w_avg_per_plot.columns] > 0).all(axis=1)]

In [ ]:
###IMPORTANT###
#USE THE CORRECT SAVE CODE ACCORDING TO THE RUN

In [ ]:
# ONE (GRASS)

df_end.to_csv('sPlot_mean_averages_per_plot_grasslands.csv', index = False)


In [ ]:
# TWO (FOREST)

df_end.to_csv('sPlot_mean_averages_per_plot_forests.csv', index = False)

In [ ]:
# THREE (SHRUBLAND)

df_end.to_csv('sPlot_mean_averages_per_plot_shrublands.csv', index = False)